In [1]:
!pip install --quiet arcgis

In [2]:
from arcgis.gis import GIS;
from arcgis.features import FeatureLayer,Table;
import pandas as pd;
import numpy as np;


In [3]:
ag_url    = 'https://gispub.epa.gov/arcgis/rest/services/OW/ATTAINS_Assessment/MapServer';
ag_points = FeatureLayer(ag_url + "/0");
ag_lines  = FeatureLayer(ag_url + "/1");
ag_areas  = FeatureLayer(ag_url + "/2");
ag_cats   = FeatureLayer(ag_url + "/3");
ag_attr   = Table(ag_url + "/4");


In [4]:
base_tbl = [
  ['AL'],['AK'],['AS'],['AZ'],['AR'],['CA'],['CO'],['CT'],['DC'],['DE'],['FL'],['GA'],['GU'],['HI'],['ID'],['IL'],
  ['IN'],['IA'],['KS'],['KY'],['LA'],['ME'],['MD'],['MA'],['MI'],['MN'],['MP'],['MS'],['MO'],['MT'],['NE'],['NV'],
  ['NH'],['NJ'],['NM'],['NY'],['NC'],['ND'],['OH'],['OK'],['OR'],['PA'],['PR'],['RI'],['SC'],['SD'],['TN'],['TX'],
  ['UT'],['VI'],['VT'],['VA'],['WA'],['WV'],['WI'],['WY']
] 
  
df = pd.DataFrame(base_tbl,columns = ['state']);
df = df.set_index('state')


In [5]:
cnt_points = ag_points.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle,organizationid,submissionid'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle,organizationid,submissionid","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle,organizationid,submissionid'
);

cnt_lines = ag_lines.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle,organizationid,submissionid'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle,organizationid,submissionid","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle,organizationid,submissionid'
);

cnt_areas = ag_areas.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle,organizationid,submissionid'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle,organizationid,submissionid","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle,organizationid,submissionid'
);

cnt_attr = ag_attr.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle,organizationid,submissionid'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle,organizationid,submissionid","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle,organizationid,submissionid'
);

cnt_cats = ag_cats.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle,organizationid,submissionid'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle,organizationid,submissionid","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle,organizationid,submissionid'
);


In [6]:
df2 = df.join(
    other = cnt_cats.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"reportingcycle":"cats_cycle","count":"cats_count"}).drop('OBJECTID',1);

df2 = df2.join(
    other = cnt_attr.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"reportingcycle":"attr_cycle","count":"attr_count"}).drop('OBJECTID',1);

df2 = df2.join(
    other = cnt_points.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"count":"point_count"}).drop('OBJECTID',1).drop('reportingcycle',1);

df2 = df2.join(
    other = cnt_lines.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"count":"line_count"}).drop('OBJECTID',1).drop('reportingcycle',1);

df2 = df2.join(
    other = cnt_areas.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"count":"area_count"}).drop('OBJECTID',1).drop('reportingcycle',1);

df2 = df2.where((pd.notnull(df2)), None);
df2

ValueError: columns overlap but no suffix specified: Index(['organizationid', 'submissionid'], dtype='object')

In [ ]:
df3 = df2;

cnt_attr_ircategory = ag_attr.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,ircategory'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,ircategory'
);
adf = cnt_attr_ircategory.sdf.pivot(index='state',columns='ircategory',values='count')
adf = adf.reset_index().rename(
    columns={
        "state":"state_delete"
       ,np.nan:"ir_cat_Null"
       ,"1":"ir_cat_1"
       ,"2":"ir_cat_2"
       ,"3":"ir_cat_3"
       ,"4A":"ir_cat_4A"
       ,"4B":"ir_cat_4B"
       ,"4C":"ir_cat_4C"
       ,"5":"ir_cat_5"
       ,"5A":"ir_cat_5A"
    }
).set_index('state_delete');

df3 = df3.join(
    other = adf
   ,how = 'left'
);

df3 = df3.where((pd.notnull(df3)), None);
df3